In [1]:
import os
import openai
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import AzureSearch
from dotenv import load_dotenv
from os.path import dirname

current_dir = os.getcwd()
root_dir = os.path.dirname(os.path.dirname(current_dir))
env_file = os.path.join(root_dir, '.env')
load_dotenv(env_file)


True

In [3]:
# Verify API Key and Endpoint
print(os.getenv("SEARCH_API_KEY"))
print(os.getenv("SEARCH_SERVICE_NAME"))
print(os.getenv('SEARCH_INDEX_NAME'))

UDShPnJq4TAtmDfa1GojvxnZgg0FPU76rAAK6NYM3VAzSeB0IH3q
https://coursera-rag-service.search.windows.net
wine-ratings-index-check


In [4]:
embeddings = OpenAIEmbeddings(deployment="text-embedding-ada-002", chunk_size=1)

# Connect to Azure AI Search
acs = AzureSearch(azure_search_endpoint=os.getenv('SEARCH_SERVICE_NAME'),
                 azure_search_key=os.getenv('SEARCH_API_KEY'),
                 index_name='wine-ratings-index-check',
                 embedding_function=embeddings.embed_query)

In [5]:
from langchain.document_loaders import CSVLoader

loader = CSVLoader(r"/workspaces/wine-ratings.csv")
documents = loader.load()

In [6]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
docs[1]

Document(page_content='name: 1000 Stories Bourbon Barrel Aged Gold Rush Red 2016\ngrape: \nregion: California\nvariety: Red Wine\nrating: 89.0\ndescription: The California Gold Rush was a period of courage, bravado and curiosity. It was with these characteristics in mind that we crafted Gold Rush Red. Grapes chosen from the golden hills and valleys of California were blended to create this bold, adventurous testament to that intrepid spirit.\nid: 1', metadata={'source': '/workspaces/wine-ratings.csv', 'row': 1})

In [ ]:
acs.add_documents(documents=docs)

In [6]:
docs = acs.similarity_search_with_relevance_scores(
    query="What is the best Cabernet Sauvignon wine in Napa Valley above 94 points",
    k=5,
)
print(docs[0][0].page_content)
print(dir(docs[0][0]))

name: Caymus Napa Valley Cabernet Sauvignon 1994
grape: 
region: Napa Valley, California
variety: Red Wine
rating: 91.0
description: #9
id: 9176
['Config', '__abstractmethods__', '__annotations__', '__class__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__rich_repr__', '__schema_cache__', '__setattr__', '__setstate__', '__signature__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__try_update_forward_refs__', '__validator

In [8]:
openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_type = "azure"
openai.api_version = "2024-02-15-preview" 
messages=[
    {"role": "system", "content": "Asisstant is a chatbot that helps you find the best wine for your taste."},
    {"role": "user", "content": "What is the best wine in Oregon above 92 points?"},
    {"role": "assistant", "content": docs[0][0].page_content}
]

response = openai.ChatCompletion.create(
    engine="gpt-4o-mini",
    messages=messages,
)
from pprint import pprint
pprint(response)
print(response['choices'][0]['message']['content'])

<OpenAIObject chat.completion id=chatcmpl-AhrsYmKhFz8X1oo5oieEsK5LqSD7g at 0x758f957d4eb0> JSON: {
  "choices": [
    {
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "protected_material_code": {
          "filtered": false,
          "detected": false
        },
        "protected_material_text": {
          "filtered": false,
          "detected": false
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      },
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Oregon is well-known for its high-quality wines, particularly Pinot Noir. Some highly rated wines in Oregon that have received scores above 92 points include